Importando as Bibliotecas 📚

In [ ]:
import pandas as pd
import numpy as np
import random
import re
import os

Tratamento dos dados: ativos financeiros (NVDA, AMZN, GOOGL, MSFT, AAPL).

In [ ]:
# Tratamento inicial dos dados:
def mudar_cabecalho(caminho_csv, ticker):
    df = pd.read_csv(caminho_csv)
    df = df.rename(columns={
        'Data': 'Date',
        'Último': f'Close_{ticker}',
        'Alta': f'High_{ticker}',
        'Baixa': f'Low_{ticker}',
        'Abertura': f'Open_{ticker}',
        'Vol.': f'Volume_{ticker}',
        'Var. %': f'Change_{ticker}'
    })
    return df

def converter_volume(valor):
    if pd.isna(valor):
        return None
    valor = valor.replace('.', '')  # Remove milhar
    valor = valor.replace(',', '.')  # Converte decimal
    if 'M' in valor:
        return int(float(valor.replace('M', '')) * 1e6)
    elif 'B' in valor:
        return int(float(valor.replace('B', '')) * 1e9)
    else:
        return int(float(valor))

def tratar_dados_acao(caminho_csv, ticker, caminho_saida):
    df = mudar_cabecalho(caminho_csv, ticker)

    # Conversão dos valores numéricos
    for col in [f'Close_{ticker}', f'High_{ticker}', f'Low_{ticker}', f'Open_{ticker}']:
        df[col] = df[col].str.replace(',', '.').astype(float)

    # Formatação da data
    df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y').dt.strftime('%d-%m-%Y')

    # Conversão do volume
    df[f'Volume_{ticker}'] = df[f'Volume_{ticker}'].apply(converter_volume)

    # Organização das colunas
    colunas_ordenadas = [
        'Date',
        f'Close_{ticker}', f'High_{ticker}', f'Low_{ticker}', f'Open_{ticker}',
        f'Volume_{ticker}', f'Change_{ticker}'
    ]
    df = df[colunas_ordenadas]

    # Ordenação cronológica
    df = df.iloc[::-1].reset_index(drop=True)

    # Salvar arquivo tratado
    df.to_csv(caminho_saida, index=False)
    print(f"Arquivo salvo: {caminho_saida}")

# Lista de ações
acoes = ['NVDA', 'AMZN', 'GOOGL', 'MSFT', 'AAPL']
pasta = '/home/boto-cdr/Área de trabalho/portfolio_github/deepseek-stock-market'

# Processar todas
for ticker in acoes:
    entrada = os.path.join(pasta, f'{ticker}.csv')
    saida = os.path.join(pasta, f'dados_{ticker}_tratados.csv')
    tratar_dados_acao(entrada, ticker, saida)


📦 1. Carregar e ordenar colunas

In [ ]:
# Load e ordenamento

# Caminho do CSV original
caminho_csv = '/deepseek-stock-market/dados_tratados.csv'

# Leitura inicial
df = pd.read_csv(caminho_csv)

# Definir a ordem das colunas
ordem_colunas = [
    "Date",
    "Close_AAPL", "Close_AMZN", "Close_GOOGL", "Close_MSFT", "Close_NVDA",
    "High_AAPL", "High_AMZN", "High_GOOGL", "High_MSFT", "High_NVDA",
    "Low_AAPL", "Low_AMZN", "Low_GOOGL", "Low_MSFT", "Low_NVDA",
    "Open_AAPL", "Open_AMZN", "Open_GOOGL", "Open_MSFT", "Open_NVDA",
    "Volume_AAPL", "Volume_AMZN", "Volume_GOOGL", "Volume_MSFT", "Volume_NVDA",
    "Change_AAPL", "Change_AMZN", "Change_GOOGL", "Change_MSFT", "Change_NVDA"
]

# Reorganizar colunas
df = df[ordem_colunas]

# Formatar datas
df["Date"] = pd.to_datetime(df["Date"], dayfirst=True).dt.strftime("%Y-%m-%d")

# Salvar versão organizada
df.to_csv('/deepseek-stock-market/dados_final.csv', index=False)


🔢 2. Garantir precisão decimal (8 casas) para preços e conversão correta de tipos

In [ ]:
# Recarregar o DataFrame para aplicar formatos
df = pd.read_csv('/deepseek-stock-market/dados_final.csv')

# Colunas de preço
price_cols = [col for col in df.columns if col.startswith(('Close_', 'High_', 'Low_', 'Open_'))]

# Converter para float, substituir vírgulas e formatar com 8 casas
for col in price_cols:
    df[col] = df[col].astype(str).str.replace(',', '.', regex=False).astype(float)
    df[col] = df[col].map(lambda x: f"{x:.8f}")

# Colunas de volume como inteiros
volume_cols = [col for col in df.columns if col.startswith('Volume_')]
for col in volume_cols:
    df[col] = df[col].astype(str).str.replace(',', '', regex=False).astype(int)

# Salvar arquivo final formatado
df.to_csv('/deepseek-stock-market/arquivo_formatado.csv', index=False)


🎯 3. Arredondamento personalizado (ex: 2 casas decimais para análise exploratória)

In [ ]:
# Reabrir CSV, por exemplo para análise
df = pd.read_csv('/deepseek-stock-market/dados_final.csv')

# Excluir 'Date' e 'Volume' das colunas a arredondar
cols_to_round = df.columns.difference(['Date'] + volume_cols)

# Arredondar para 2 casas decimais (exibição e análise)
df[cols_to_round] = df[cols_to_round].astype(float).round(2)

# Exibir com float-format pandas
pd.options.display.float_format = '{:.2f}'.format

# Exportar versão arredondada
df.to_csv('/deepseek-stock-market/dados_arredondados.csv', index=False, float_format='%.2f')


🧪 4. Adicionar decimais aleatórios extras (para simulação, diferencial ou ruído)

In [ ]:
def add_decimal_extra(value_str):
    val = value_str.strip('"').replace(',', '.')
    try:
        base = float(val)
    except:
        return value_str  # Volumes ou datas

    # Criar decimais falsos adicionais
    extra_decimals = ''.join(str(random.randint(1, 9)) for _ in range(15))
    base_str = f"{base:.8f}".rstrip('0').rstrip('.')

    if '.' in base_str:
        inteiro, decimal = base_str.split('.')
        novo_valor_str = f"{inteiro}.{decimal}{extra_decimals}"
    else:
        novo_valor_str = f"{base_str}.{extra_decimals}"

    return novo_valor_str

def process_csv(input_file, output_file):
    df = pd.read_csv(input_file, dtype=str)
    cols_to_modify = [col for col in df.columns if re.match(r'(Close_|High_|Low_|Open_)', col)]
    for col in cols_to_modify:
        df[col] = df[col].apply(add_decimal_extra)
    df.to_csv(output_file, index=False)
    print(f"Arquivo processado salvo em: {output_file}")

# Executar com caminho
input_csv = '/deepseek-stock-market/dados_final.csv'
output_csv = '/deepseek-stock-market/dados_modificado.csv'
process_csv(input_csv, output_csv)


✅ 5.Exibir visualmente ou trabalhar com uma versão para plotagem

In [ ]:
# Carregar e preparar uma cópia para visualizações
df_plot = df.copy()

# Selecionar colunas relevantes para gráfico (excluindo volumes)
cols_to_plot = [col for col in df_plot.columns if col.startswith(('Close_', 'High_', 'Low_', 'Open_'))]

# Converter para float se necessário
for col in cols_to_plot:
    df_plot[col] = df_plot[col].astype(str).str.replace(',', '.', regex=False).astype(float)
    df_plot[col] = df_plot[col].round(4)  # Pode ajustar para 2 ou 8 conforme necessário

print(df_plot.head())


🧼 6. Remoção de valores ausentes (NaN)

In [ ]:
# Recarregar o DataFrame
df = pd.read_csv('/deepseek-stock-market/arquivo_formatado.csv')

# Verificar presença de NaNs
print("Total de valores nulos por coluna:")
print(df.isnull().sum())

# Estratégia simples: remover linhas com qualquer NaN
df = df.dropna()

# Alternativa: preencher valores ausentes (ex: com valor anterior)
# df.fillna(method='ffill', inplace=True)

# Salvar versão limpa
df.to_csv('/deepseek-stock-market/dados_sem_nan.csv', index=False)


🔁 7. Verificar e remover duplicatas

In [ ]:
# Verificar se existem linhas duplicadas
duplicatas = df.duplicated()
print(f"Número de linhas duplicadas: {duplicatas.sum()}")

# Remover duplicadas
df = df.drop_duplicates()

# Salvar versão sem duplicatas
df.to_csv('/deepseek-stock-market/dados_sem_duplicatas.csv', index=False)


🧹 8. Verificar consistência e formato de colunas financeiras

In [ ]:
# Validar se todas as colunas numéricas têm o formato correto
for col in df.columns:
    if col.startswith(('Close_', 'Open_', 'High_', 'Low_')):
        try:
            df[col] = df[col].astype(float)
        except Exception as e:
            print(f"Erro ao converter a coluna {col}: {e}")

# Verificar se volumes são inteiros positivos
for col in df.columns:
    if col.startswith('Volume_'):
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df = df[df[col] >= 0]  # Remover registros com volume negativo

# Salvar versão validada
df.to_csv('/deepseek-stock-market/dados_validados.csv', index=False)
